# Mass balance of all glaciers: Maps

## Import packages

In [ ]:
import fsspec
import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import xarray as xr
import xskillscore as xs
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
period_start = "1975_1976"
period_stop = "2020_2021"
assert all("_" in period and len(period) == 9 for period in (period_start, period_stop))

## Define request

In [ ]:
y0_start, y1_start = map(int, period_start.split("_"))
y0_stop, y1_stop = map(int, period_stop.split("_"))
collection_id = "derived-gridded-glacier-mass-change"
request = {
    "variable": "glacier_mass_change",
    "product_version": "wgms_fog_2022_09",
    "format": "zip",
    "hydrological_year": [
        f"{y0}_{str(y1)[-2:]}"
        for y0, y1 in zip(range(y0_start, y0_stop + 1), range(y1_start, y1_stop + 1))
    ],
}

## Functions to cache

In [ ]:
def compute_coeff_and_pvalue(cumsum, degree):
    coeff = cumsum.polyfit("time", degree)
    (fit,) = xr.polyval(cumsum["time"], coeff).data_vars.values()
    p_value = xs.pearson_r_p_value(cumsum.chunk(time=-1), fit.chunk(time=-1), "time")
    return coeff, p_value


def compute_time_statistics(ds):
    with xr.set_options(keep_attrs=True):
        cumsum = ds["Glacier"].cumsum("time").drop_vars("time")
        ds["Uncertainty"] = (ds["Uncertainty"] ** 2).sum("time") ** (1 / 2)

    # Sum
    ds = ds.sum("time", keep_attrs=True)
    for da in ds.data_vars.values():
        da.attrs["long_name"] = f"Sum of {da.attrs['long_name']}"

    # Linear
    coeff, p_value = compute_coeff_and_pvalue(cumsum, 1)
    ds["Slope"] = coeff["polyfit_coefficients"].sel(degree=1)
    ds["Slope"].attrs = {
        "long_name": f"Trend of {da.attrs['long_name']}",
        "units": f"{da.attrs['units']} yr$^{-1}$",
    }
    ds["Pvalue1"] = p_value
    ds["Pvalue1"].attrs = {
        "long_name": f"Linear p-value of {da.attrs['long_name']}",
    }

    # Quadratic
    coeff, p_value = compute_coeff_and_pvalue(cumsum, 2)
    ds["Acceleration"] = 2 * coeff["polyfit_coefficients"].sel(degree=2)
    ds["Acceleration"].attrs = {
        "long_name": f"Quadratic Trend of {da.attrs['long_name']}",
        "units": f"{da.attrs['units']} yr$^{-2}$",
    }
    ds["Pvalue2"] = p_value
    ds["Pvalue2"].attrs = {
        "long_name": f"Quadratic p-value of {da.attrs['long_name']}",
    }

    return ds


def compute_spatial_statistics(ds):
    with xr.set_options(keep_attrs=True):
        return ds.sum(("latitude", "longitude"))

## Download and transform data

In [ ]:
chunks = {"hydrological_year": 1}
ds = download.download_and_transform(
    collection_id,
    request,
    chunks=chunks,
    transform_func=compute_time_statistics,
    transform_chunks=False,
)
ds_timeseries = download.download_and_transform(
    collection_id,
    request,
    chunks=chunks,
    transform_func=compute_spatial_statistics,
)

# Customize some attributes
for obj in (ds, ds_timeseries):
    for da in obj.data_vars.values():
        da.attrs["long_name"] = da.attrs["long_name"].replace("_", " ").title()

## Create geodataframe

In [ ]:
# Maps
ds_stack = ds.stack(latlon=("latitude", "longitude"))
ds_stack = ds_stack.where((ds_stack["Glacier"] != 0).compute(), drop=True)
df = ds_stack.to_pandas()
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326",
)

## Plotting function

In [ ]:
def plot_map(gdf, var_name=None, label=None, title=None, **kwargs):
    kwargs = {"markersize": 5, "legend": var_name is not None} | kwargs
    if var_name:
        kwargs = {"c": var_name, "column": var_name} | kwargs
        kwargs.setdefault("legend_kwds", {"shrink": 0.49, "extend": "both"})
        if label is not None:
            kwargs["legend_kwds"].setdefault("label", label)

    shapefile_url = "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
    with fsspec.open(f"simplecache::{shapefile_url}") as f:
        gdf_countries = gpd.read_file(f)

    ax = gdf_countries.boundary.plot(
        figsize=(20, 16), facecolor="none", edgecolor="black", linewidth=0.25
    )
    gdf.plot(ax=ax, **kwargs)
    ax.axis("off")
    if title:
        ax.set_title(title, fontsize=25)
    return ax

## Plot maps

In [ ]:
column_kwargs = {
    "Glacier": {
        "cmap": "coolwarm_r",
        "norm": mcolors.TwoSlopeNorm(vmin=-5, vcenter=0, vmax=2),
        "label": "Cumulative mass change (Gt)",
        "title": "Cumulative glacier mass change",
    },
    "Uncertainty": {
        "cmap": "coolwarm",
        "vmin": 0,
        "vmax": 5,
        "label": "Total mass change error (Gt)",
        "title": "Sum of glacier mass change error",
    },
    "Slope": {
        "cmap": "coolwarm_r",
        "vmin": -0.1,
        "vmax": +0.1,
        "label": "Mass change trend (Gt yr$^{-1}$)",
        "title": "Linear trends (slopes) of the mass changes of the glaciers.",
    },
    "Pvalue1": {
        "title": "Linear p-value.",
    },
    "Acceleration": {
        "cmap": "coolwarm_r",
        "vmin": -0.005,
        "vmax": +0.005,
        "label": "Total mass change error (Gt yr$^{-2}$)",
        "title": "Quadratic trends (accelerations) of the mass changes of the glaciers.",
    },
    "Pvalue2": {
        "title": "Quadratics p-value.",
    },
}
for var_name, kwargs in column_kwargs.items():
    ax = plot_map(gdf, var_name, **kwargs)
    plt.show()

## Plot timeseries

In [ ]:
for da in ds_timeseries.data_vars.values():
    da.plot()
    plt.grid()
    plt.show()